<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install presidio_analyzer
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.0 MB/s eta 0:00:00


In [2]:
from presidio_analyzer import AnalyzerEngine
from faker import Faker

analyzer=AnalyzerEngine()
# ent=analyzer.get_supported_entities()
# for e in ent:
#   print(e)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import pandas as pd

def analyze_column(df):
  entity_columns = {}  # Initialize as a dictionary
  for col in df.columns:
    unique_values = df[col].dropna().astype(str).unique()[:25]
    entity_counts = {}

    for value in unique_values:
      results = analyzer.analyze(text=value, language="en")
      for result in results:
        entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1
    if entity_counts:
      predominant_entity = max(entity_counts, key=entity_counts.get)
      if predominant_entity not in entity_columns:
        entity_columns[predominant_entity] = []  # Assign a list to the entity key
      entity_columns[predominant_entity].append(col)
  return entity_columns  # Return the dictionary


def load_and_analyze(file_path):
  if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, engine="python")
  elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path)
  else:
    raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

  df_sample = df.drop_duplicates().head()
  classifeid_columns = analyze_column(df_sample)
  return classifeid_columns  # Return the dictionary

In [9]:
file="split_file_part1.csv"
classifeid_columns=load_and_analyze(file)
df=pd.read_csv(file)
print(classifeid_columns)
# masked_df=anonymize_data(df,classifeid_columns)
# print(masked_df)
# anonymize_csv(file,"masked.csv",classifeid_columns)

{'DATE_TIME': ['Unnamed: 0', 'size range'], 'LOCATION': ['name', 'locality', 'country'], 'URL': ['domain', 'linkedin url'], 'US_DRIVER_LICENSE': ['current employee estimate', 'total employee estimate']}


In [14]:
import json
import os
import pandas as pd
from faker import Faker

fake = Faker()
Faker.seed(42)

MAPPING_FILE = "output_xlsx/data_mapping.json"
forward_mapping = {}
reverse_mapping = {}

def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                # Convert keys back to tuples if needed
                forward_mapping = {tuple(eval(k)): v for k, v in data.get("forward_mapping", {}).items()}
                reverse_mapping = {tuple(eval(k)): v for k, v in data.get("reverse_mapping", {}).items()}
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            # If the file is invalid, reset the mappings
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    os.makedirs(os.path.dirname(MAPPING_FILE), exist_ok=True)
    # Convert tuple keys to strings before saving
    modified_forward_mapping = {str(k): v for k, v in forward_mapping.items()}
    modified_reverse_mapping = {str(k): v for k, v in reverse_mapping.items()}
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": modified_forward_mapping, "reverse_mapping": modified_reverse_mapping}, f, indent=4)



def generate_fake_value(original: str, entity_type: str) -> str:
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        # "DATE": fake.date_of_birth,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict) -> pd.DataFrame:
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type) if x.strip() else x)
    return df

if __name__ == "__main__":
    load_mapping()
    input_file = "split_file_part1.csv"
    output_file = "output_xlsx/anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze("split_file_part1.csv")

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)
    save_mapping()

In [55]:
faker_mapping = {
    "PERSON": fake.name,
    "FIRST_NAME": fake.first_name,
    "LAST_NAME": fake.last_name,
    "EMAIL": fake.email,
    "URL": fake.url,
    "PHONE_NUMBER": fake.phone_number,
    "CREDIT_CARD": fake.credit_card_number,
    "IBAN": fake.iban,
    "US_SSN": fake.ssn,
    # "DATE": fake.date_of_birth,
    "LOCATION": fake.address,
    "STREET_ADDRESS": fake.street_address,
    "CITY": fake.city,
    "STATE": fake.state,
    "COUNTRY": fake.country,
    "ZIP_CODE": fake.zipcode,
    "ORGANIZATION": fake.company,
    "JOB_TITLE": fake.job,
    "USERNAME": fake.user_name,
    "PASSWORD": fake.password,
    "IP_ADDRESS": fake.ipv4,
    "MAC_ADDRESS": fake.mac_address,
    "LICENSE_PLATE": fake.license_plate,
    "UUID": fake.uuid4,
    "BANK_ACCOUNT": fake.bban,
    "TRANSACTION_ID": fake.uuid4,
    "DEVICE_ID": fake.uuid4,
}

KeyError: 'Name'